In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly
import plotly.graph_objects as go

from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

import datetime

#DataFrame
df = pd.read_csv('owid-covid-data.csv')
df = df.fillna(0)
w = df[df['location'] == 'World']
us = df[df['location'] == 'United States']
ind = df[df['location'] == 'India']
brz = df[df['location'] == 'Brazil']
rus = df[df['location'] == 'Russia']
uk = df[df['location'] == 'United Kingdom']
fr = df[df['location'] == 'France']
sp = df[df['location'] == "Spain"]
gr = df[df['location'] == "Germany"]
cn = df[df['location'] == 'China']
th = df[df['location'] == 'Thailand']
vn = df[df['location'] == 'Vietnam']
ml = df[df['location'] == 'Malaysia']
jp = df[df['location'] == 'Japan']
sk = df[df['location'] == 'South Korea']
pl = df[df['location'] == 'Philippines']
idn = df[df['location'] == 'Indonesia']
sg = df[df['location'] == 'Singapore']
isr = df[df['location'] == 'Israel']
au = df[df['location'] == 'Australia']
countries = [w,us,ind,brz,rus,uk,fr,sp,gr,cn,th,vn,ml,jp,sk,pl,sg,isr,au,idn]

In [ ]:
for country in countries:
    country.fillna(0)
    country = country.reset_index()
    
    #Loop through to find the degree of polynomial that best fits 
    for i in range(5,12):
        #Preprocessing data and Fitting using linear regression
        x = np.array(country.index).reshape(-1,1)
        y = np.array(country.new_cases).reshape(-1,1)
        
        polyFeat = PolynomialFeatures(degree=i)
        x = polyFeat.fit_transform(x)
        model = linear_model.LinearRegression()
        model.fit(x,y)
        accuracy = model.score(x,y)
        print("Accuracy: ",accuracy)
        
        y0 = model.predict(x)
        yx = pd.DataFrame(y0,columns=['New_cases'])
        
        #Prediction (496 days since the first record) after 1 week, 1 month and 3 months
        x1w = np.array(list(range(0,496+7))).reshape(-1,1)
        x1m = np.array(list(range(0,496+30))).reshape(-1,1)
        x3m = np.array(list(range(0,496+90))).reshape(-1,1)
        y1w = model.predict(polyFeat.fit_transform(x1w))
        y1m = model.predict(polyFeat.fit_transform(x1m))
        y3m = model.predict(polyFeat.fit_transform(x3m))
        y1w = pd.DataFrame(y1w,columns=['New_cases'])
        y1m = pd.DataFrame(y1m,columns=['New_cases'])
        y3m = pd.DataFrame(y3m,columns=['New_cases'])

        #Plotting graphs
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=pd.date_range(start = country.date.iloc[0],end = '2021-5-31',freq='D'),
                                 y=country.new_cases,mode='markers',
                                 name="New cases",marker_symbol='circle-open'))
        
        #fig.add_trace(go.Scatter(x=pd.date_range(start = country.date.iloc[0],end = '2021-5-31',freq='D'),
                                 #y=country.new_vaccinations/50,mode='markers',
                                 #name='New Vaccination divide by 50 units',marker_symbol='circle-open'))
        
        fig.add_trace(go.Scatter(x=pd.date_range(start = country.date.iloc[0],end = '2021-5-31',freq='D'),
                                 y=country.total_vaccinations/1000,mode='markers',
                                 name='Total Vaccination in thousand unit',marker_symbol='circle-open'))
        fig.add_trace(go.Scatter(x=pd.date_range(start = country.date.iloc[0],end = '2021-9-1',freq='D'),
                                 y=y3m.New_cases,mode='markers',
                                 name='Prediction in 3 monnths',
                                 marker_symbol='star'))
        fig.add_trace(go.Scatter(x=pd.date_range(start = country.date.iloc[0],end = '2021-7-1',freq='D'),
                                 y=y1m.New_cases,mode='markers',
                                 name='Prediction in 1 monnth',
                                 marker_symbol='star'))
        fig.add_trace(go.Scatter(x=pd.date_range(start = country.date.iloc[0],end = '2021-6-8',freq='D'),
                                 y=y1w.New_cases,mode='markers',
                                 name='Prediction in 1 week',
                                 marker_symbol='star'))
        
        
        
        #countryname = str(country.iloc[0].location) + " " + str(i)
        #print(countryname)
        fig.update_layout(yaxis_title='Cases',title=country.iloc[0].location)
        #fig.write_html('html_folder/data_%s.html' % countryname) 
        #fig.write_image("html_folder/image/data_%s.png" % countryname)
        fig.show()